In [2]:
import pandas as pd 
import numpy as np
import pyecharts
import plotly.express as px # visualization
import pandas_datareader as pdr # access fred
from datetime import date
from fredapi import Fred
import requests
import openpyxl
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
import re

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

from pyecharts import options as opts
from pyecharts.charts import Bar, Line

In [124]:
# Load orginal dataset

index_data = pd.read_excel('C:/Users/fanjz/Desktop/python/nmi/nmi_level.xlsx')
glance = pd.read_excel('C:/Users/fanjz/Desktop/python/nmi/glancetable.xlsx')
comments = pd.read_excel('C:/Users/fanjz/Desktop/python/nmi/comments.xlsx')
sector_ranking = pd.read_excel('C:/Users/fanjz/Desktop/python/nmi/sector_ranking.xlsx')


In [45]:
# Copy and paste latest month data url

url = 'https://www.prnewswire.com/news-releases/services-pmi-at-53-6-september-2023-services-ism-report-on-business-301946224.html'

response = requests.get(url)

### Extract Data from URL ###

#### 1. Get comments from respondents ####

In [46]:
# Extract What Respondents Say

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')   
    section = soup.find('section', class_='release-body container')

    if section:
        main_content = section.find('div', class_='col-lg-10 col-lg-offset-1')

        if main_content:
            ul_element = main_content.find('ul', type='disc')
            if ul_element:
                list_items = ul_element.find_all('li')
                if list_items:
                    # Create a list to store the text of each list item
                    items_list = [item.text.strip() for item in list_items]

                    # Create a DataFrame from the list
                    respondent_comment = pd.DataFrame(items_list, columns=['WHAT RESPONDENTS ARE SAYING'])
        else:
            print("Main content not found within the section.")
    else:
        print("Section with class 'xxxx' not found.")

else:
    print("Failed to retrieve the page")

#### 2. Get Monthly Level Data ####

In [22]:
# Extract monthly index level

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')   
    section = soup.find('section', class_='release-body container')

    if section:
        table_div = section.find('div', class_='table-responsive')

        if table_div:
        # Find the table within the div
            table = table_div.find('table')

            if table:
                # Read the table into a DataFrame
                data_table = pd.read_html(str(table))[0]
            else:
                print("Table not found within the div with class 'table-responsive'")
        else:
            print("Div with class 'table-responsive' not found.")
    else:
        print("Failed to retrieve the page")

C:\Users\fanjz\AppData\Local\Temp\ipykernel_15908\2319469250.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_table = pd.read_html(str(table))[0]


#### 3. Get Commodity Data ####

In [24]:
# Extract Commodity up/down/short supply

commodity_up = soup.select_one('#main > article > section > div > div > p:nth-child(14)')
commodity_down = soup.select_one('#main > article > section > div > div > p:nth-child(15)')
commodity_short = soup.select_one('#main > article > section > div > div > p:nth-child(16)')

# commodity_up = soup.select_one('#main > article > section > div > div > p:nth-child(16)')
# commodity_down = soup.select_one('#main > article > section > div > div > p:nth-child(17)')
# commodity_short = soup.select_one('#main > article > section > div > div > p:nth-child(18)')

In [25]:
commodity_tags = [commodity_up,commodity_down,commodity_short]
commoditiy_direction = []
for i, tag in enumerate(commodity_tags):
    if tag:
        direction = tag.get_text(separator=' ').strip()  # Get all the text within the <p> tag
        commoditiy_direction.append(direction)
    else:
        print(f"The specified <p> tag {i+1} was not found.")

# Create a DataFrame with one column and three rows from the list of contents
commodity_comments = pd.DataFrame(commoditiy_direction, columns=['content'])

In [26]:
commodity_comments

,content
0,Commodities Up in Price Diesel Fuel (3); Elect...
1,Commodities Down in Price Steel Products (2).
2,Commodities in Short Supply Construction Mater...


In [27]:
substring_patterns = ['Commodities Up in Price','Commodities Down in Price','Commodities in']

def split_based_on_pattern(content, pattern):
    if pattern in content:
        parts = content.split(pattern, 1)
        return [pattern.strip(), parts[1].strip()]
    else:
        return [content.strip(), '']

# Split each row based on the defined patterns
split_data = []
for i, row in commodity_comments.iterrows():
    split_data.append(split_based_on_pattern(row['content'], substring_patterns[i]))

# Create a new DataFrame from the split data
commodity_df = pd.DataFrame(split_data, columns=['pattern', 'rest'])

# Display the split DataFrame
commodity_df

,pattern,rest
0,Commodities Up in Price,Diesel Fuel (3); Electrical Equipment; Electri...
1,Commodities Down in Price,Steel Products (2).
2,Commodities in,Short Supply Construction Materials; Electrica...


#### 4. Get Industry Ranking #### 

In [52]:
# Extract Industry Rankings

# pmi = soup.select_one('#main > article > section > div > div > p:nth-child(10)')
# new_order = soup.select_one('#main > article > section > div > div > p:nth-child(26)')
# production = soup.select_one('#main > article > section > div > div > p:nth-child(29)')
# employment= soup.select_one('#main > article > section > div > div > p:nth-child(32)')
# supplier_deliveries = soup.select_one('#main > article > section > div > div > p:nth-child(35)')
# inventories = soup.select_one('#main > article > section > div > div > p:nth-child(38)')
# customers_inv = soup.select_one('#main > article > section > div > div > p:nth-child(41)')
# prices = soup.select_one('#main > article > section > div > div > p:nth-child(44)')
# backlog_orders = soup.select_one('#main > article > section > div > div > p:nth-child(47)')
# new_export_orders = soup.select_one('#main > article > section > div > div > p:nth-child(50)')
# imports = soup.select_one('#main > article > section > div > div > p:nth-child(53)')

nmi = soup.select_one('#main > article > section > div > div > p:nth-child(8)')
businesss_activity = soup.select_one('#main > article > section > div > div > p:nth-child(24)')
new_order = soup.select_one('#main > article > section > div > div > p:nth-child(27)')
employment= soup.select_one('#main > article > section > div > div > p:nth-child(30)')
supplier_deliveries = soup.select_one('#main > article > section > div > div > p:nth-child(33)')
inventories = soup.select_one('#main > article > section > div > div > p:nth-child(36)')
prices = soup.select_one('#main > article > section > div > div > p:nth-child(39)')
backlog_orders = soup.select_one('#main > article > section > div > div > p:nth-child(43)')
new_export_orders = soup.select_one('#main > article > section > div > div > p:nth-child(46)')
imports = soup.select_one('#main > article > section > div > div > p:nth-child(49)')
inventory_sentiment = soup.select_one('#main > article > section > div > div > p:nth-child(52)')

In [ ]:
#main > article > section > div > div > p:nth-child(8)
#main > article > section > div > div > p:nth-child(24)
#main > article > section > div > div > p:nth-child(43)

In [53]:
industry_ranking = [nmi,businesss_activity,new_order,employment,supplier_deliveries,inventories,prices,backlog_orders,new_export_orders,imports, inventory_sentiment]

rankings = []
for i, tag in enumerate(industry_ranking):
    if tag:
        ranking = tag.get_text().strip()  # Get all the text within the <p> tag
        rankings.append(ranking)
    else:
        print(f"The specified <p> tag {i+1} was not found.")

# Create a DataFrame with one column and three rows from the list of contents
ranking_df = pd.DataFrame(rankings, columns=['Ranking'])
ranking_df['Ranking'] = ranking_df['Ranking'].str.strip()


In [54]:
industry_list = ['NMI','Business Activity','New Order','Employment','Supplier Deliveries','Inventories','Prices','Backlog Orders','New Export Orders','Imports','Inventory Sentiment']
ranking_df['Industry'] = industry_list
col_oder = ['Industry','Ranking']
ranking_df = ranking_df[col_oder]

In [55]:
# Data pre-check
pre_check_dfs = [respondent_comment, data_table, commodity_df, ranking_df]
ranking_df

,Industry,Ranking
0,NMI,INDUSTRY PERFORMANCEThe 13 services industries...
1,Business Activity,The 14 industries reporting an increase in bus...
2,New Order,The 12 industries reporting an increase in new...
3,Employment,The nine industries reporting an increase in e...
4,Supplier Deliveries,The six industries reporting slower deliveries...
5,Inventories,The nine industries reporting an increase in i...
6,Prices,Twelve services industries reported an increas...
7,Backlog Orders,The six industries reporting an increase in or...
8,New Export Orders,The nine industries reporting an increase in n...
9,Imports,The three industries reporting an increase in ...


### Working on Data Clenaing ### 

##### 1. PMI Index Level #####

In [83]:
# variables = data_table, respondent_comment, commodity_df, ranking_df
# 1. Working on data_table

new_table = data_table.iloc[2:]
header = new_table.iloc[0]
final_table = new_table.iloc[1:]
final_table.columns = header

In [89]:
new_month_level = final_table.iloc[:,1]
new_month_level = new_month_level[:-3]
new_month_level = new_month_level.to_frame().T

In [91]:
latest_date = index_data['Date'].max()
new_date = latest_date + relativedelta(months=1)
new_date_str = new_date.strftime('%Y-%m-%d')
new_month_level["Date"] = new_date
new_month_level = new_month_level[['Date'] + [col for col in new_month_level if col != 'Date']]
new_month_level.columns = index_data.columns

updated_index_data = pd.concat([index_data, new_month_level], ignore_index=True)

##### 2. What Respondents Saying #####

In [93]:
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 150)
respondent_comment

,WHAT RESPONDENTS ARE SAYING
0,"""Prices are coming down across the board for most commodities. However, there are still a few areas where supply is not available on a consistent ..."
1,"""Conditions remain favorable for mechanical contractors. New construction projects continue to launch. We are still seeing opportunities for cost ..."
2,"""The market is stable at this time."" [Educational Services]"
3,"""I think the outlook of our company and the industry is a slow but steady improvement — from a plethora of unknowns to a daily sense of being able..."
4,"""The market for the industry keeps looking positive, although there is a higher concentration of new projects at emerging markets. Opportunities i..."
5,"""Bank and leasing company volume seems to be falling as credit tightens, thus causing a slowdown in related services industries. Bankruptcy work i..."
6,"""The fourth quarter is looking better than forecast, which is good because the third quarter of 2023 was below plan. Our customers are cautiously ..."
7,"""Business is ramping up in preparation for the holiday season. Our supply chain is strong."" [Retail Trade]"
8,"""Higher level of orders in past month. Business activity is stabilizing to last year's numbers."" [Transportation & Warehousing]"
9,"""Other than increases in fuel costs and some materials, activity and sales have been relatively stable month over month. However, a seasonal decre..."


In [94]:
updated_comments = respondent_comment['WHAT RESPONDENTS ARE SAYING'].str.split('[', expand=True)
updated_comments.columns = [new_date_str, 'Industry']
updated_comments['Industry'] = updated_comments['Industry'].str.replace(']', '', regex=False)
updated_comments[new_date_str] = updated_comments[new_date_str].str.replace('"', '', regex=False)
updated_comments = updated_comments.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

col_order = ['Industry',new_date_str]
updated_comments = updated_comments[col_order]

In [95]:
industry_col1 = comments['Industry']
industry_col2 = updated_comments['Industry']

merged_df = pd.concat([industry_col1, industry_col2], ignore_index=True)
merged_series = merged_df.squeeze().drop_duplicates()
merge_comment = merged_series.to_frame(name='Industry')

In [96]:
merge_final_df1 = pd.merge(merge_comment, comments, on='Industry', how='left')
final_comment = pd.merge(merge_final_df1, updated_comments, on='Industry', how='left')

##### 3. Commodity Product Pricing #####

In [97]:
commodity_df

commodity_dfs = []

for index, row in commodity_df.iterrows():
    # Create a new DataFrame for the current row
    row_df = pd.DataFrame([row])
    # Append the new DataFrame to the list
    commodity_dfs.append(row_df)
    

In [98]:
commodity_df

,pattern,rest
0,Commodities Up in Price,Diesel Fuel (3); Electrical Equipment; Electrical Components (32); Fuel (2); Gasoline (8); Labor (34); Labor — Contract; Labor — Services; Lumber ...
1,Commodities Down in Price,Steel Products (2).
2,Commodities in,"Short Supply Construction Materials; Electrical Components (6); Heating, Ventilation and Air Conditioning (HVAC) Equipment (2); Labor (11); Labor ..."


In [99]:
commodity_up_df = commodity_dfs[0]
commodity_up_df_split = commodity_up_df['rest'].str.split(';', expand = True).T
commodity_up_header = ['Commodities Up in Price']
commodity_up_df_split.columns = commodity_up_header


commodity_down_df = commodity_dfs[1]
commodity_down_df_split = commodity_down_df['rest'].str.split(';', expand = True).T
commodity_down_header = ['Commodities Down in Price']
commodity_down_df_split.columns = commodity_down_header

commodity_short_df = commodity_dfs[2]
commodity_short_df_split = commodity_short_df['rest'].str.split(';', expand = True).T
commodity_short_header = ['Commodities in Short Supply']
commodity_short_df_split.columns = commodity_short_header

commodity_up_df

In [101]:
commodity_up_df_split[['Commodity Price Up', 'Number of Month']] = commodity_up_df_split['Commodities Up in Price'].str.split("(", n=1, expand=True)
commodity_up_df_split['Commodity Price Up'] = commodity_up_df_split['Commodity Price Up'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
commodity_up_df_split['Number of Month'] = commodity_up_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
commodity_up_df_split = commodity_up_df_split.drop(columns=['Commodities Up in Price'])

commodity_down_df

In [102]:
if '(' in commodity_down_df_split:
    commodity_down_df_split[['Commodity Price Down', 'Number of Month']] = commodity_down_df_split['Commodities Down in Price'].str.split("(", n=1, expand=True)
    commodity_down_df_split['Commodity Price Down'] = commodity_down_df_split['Commodity Price Down'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
    commodity_down_df_split['Number of Month'] = commodity_down_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
    commodity_down_df_split = commodity_down_df_split.drop(columns=['Commodities Down in Price'])
else:
    commodity_down_df_split['Number of Month'] = [1]
    commodity_down_df_split.rename(columns={'Commodities Down in Price': 'Commodity Price Down'})

Commodity in short supply

In [103]:
commodity_short_df_split[['Commodities: Short Supply', 'Number of Month']] = commodity_short_df_split['Commodities in Short Supply'].str.split("(", n=1, expand=True)
commodity_short_df_split['Commodities: Short Supply'] = commodity_short_df_split['Commodities: Short Supply'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
commodity_short_df_split['Number of Month'] = commodity_short_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
commodity_short_df_split = commodity_short_df_split.drop(columns=['Commodities in Short Supply'])
commodity_short_df_split['Commodities: Short Supply'] = commodity_short_df_split['Commodities: Short Supply'].str.replace('Short Supply ', '', regex=True)

Commodity Data Final Output

In [104]:
commodity_short_df_split = commodity_short_df_split.sort_values(['Number of Month'])
commodity_up_df_split = commodity_up_df_split.sort_values(['Number of Month'])

if (commodity_down_df_split['Number of Month'] > 1).any():
    commodity_up_df_split = commodity_up_df_split.sort_values(['Number of Month'])
else:
    commodity_down_df_split

##### 4. Industry Ranking #####

In [105]:
ranking_df['Ranking'] = ranking_df['Ranking'].str.strip().str.replace('—','',regex = False).str.replace(',','',regex = False)
ranking_df_split = ranking_df['Ranking'].str.split('.', expand = True)
ranking_df_split.columns = ['Industry Up','Industry Down','No Change','None']

In [106]:
ranking_up_df = pd.concat([ranking_df['Industry'], ranking_df_split['Industry Up']], axis=1)
ranking_down_df = pd.concat([ranking_df['Industry'], ranking_df_split['Industry Down']], axis=1)

In [107]:
ranking_up = ranking_up_df['Industry Up'].str.split(':', expand=True)
ranking_down = ranking_down_df['Industry Down'].str.split(':', expand=True)

col_name = ['split_col','ranking_col']
ranking_up.columns = col_name
ranking_down.columns = col_name

In [108]:
filter_is_up = ranking_up['split_col'].str.split('is ', expand=True)
filter_is_down = ranking_down['split_col'].str.split('is ', expand=True)

In [109]:
filter_is_up = pd.concat([ranking_df['Industry'], filter_is_up], axis=1)
filter_is_down = pd.concat([ranking_df['Industry'], filter_is_down], axis=1)

In [110]:
def select_columns(df):
    if len(df.columns) >= 3:  # Check if DataFrame has at least 3 columns
        return df.iloc[:, [0, 2]]  # Select first and third columns
    else:
        return df.iloc[:, [0]]  # Select only the first column

filter_is_up_df = select_columns(filter_is_up)
filter_is_down_df = select_columns(filter_is_down)

In [111]:
ranking_up_final = pd.concat([ranking_df['Industry'], ranking_up['ranking_col']], axis=1)
ranking_down_final = pd.concat([ranking_df['Industry'], ranking_down['ranking_col']], axis=1)

In [112]:
ranking_up_final = pd.merge(ranking_up_final,filter_is_up_df, on= 'Industry')
ranking_down_final = pd.merge(ranking_down_final,filter_is_down_df, on= 'Industry')

In [113]:
def merge_columns(df):
    if len(df.columns) >= 3:
        df['Combined'] = df.iloc[:, 1].fillna(df.iloc[:, 2])
        df.drop(['ranking_col', 1], axis=1, inplace=True)
        return df
    else:
        df.rename(columns={'ranking_col': 'Combined'}, inplace=True)
        return df

ranking_up_final = merge_columns(ranking_up_final)
ranking_down_final = merge_columns(ranking_down_final)

In [114]:
ranking_up_final = ranking_up_final['Combined'].str.split(';', expand=True)
ranking_down_final = ranking_down_final['Combined'].str.split(';', expand=True)

In [115]:
ranking_up_final = ranking_up_final.map(lambda x: x.replace('and', '').strip() if isinstance(x, str) else x)
ranking_down_final = ranking_down_final.map(lambda x: x.replace('and', '').strip() if isinstance(x, str) else x)

In [116]:
ranking_up_final = pd.concat([ranking_df['Industry'], ranking_up_final], axis=1)
ranking_down_final = pd.concat([ranking_df['Industry'], ranking_down_final], axis=1)

In [117]:
rakning_up_column_order = [ranking_up_final.columns[0]] + list(range(1, len(ranking_up_final.columns)))
rakning_down_column_order = [ranking_down_final.columns[0]] + list(range(-1, -len(ranking_down_final.columns), -1))

ranking_up_final.columns = rakning_up_column_order
ranking_down_final.columns = rakning_down_column_order

In [118]:
ranking_up_df = pd.melt(ranking_up_final, id_vars=['Industry'],var_name='myVarname', value_name='myValname')
ranking_down_df = pd.melt(ranking_down_final, id_vars=['Industry'],var_name='myVarname', value_name='myValname')

In [119]:
industry_ranking_final = pd.concat([ranking_up_df, ranking_down_df],ignore_index=True)

industry_ranking_final = industry_ranking_final.rename(columns={'Industry': 'Index','myVarname': new_date_str, 'myValname': 'Industry'})

new_col_order = ['Index','Industry',new_date_str]
industry_ranking_final = industry_ranking_final[new_col_order]


In [125]:
desired_order = ranking_df['Industry'].to_list()

industry_ranking_final['Index'] = pd.Categorical(industry_ranking_final['Index'], categories=desired_order, ordered=True)
industry_ranking_final = industry_ranking_final.sort_values(['Index',new_date_str])
latest_industry_ranking = industry_ranking_final.dropna(subset=['Industry'])

sector_ranking_final = pd.merge(sector_ranking, latest_industry_ranking, on=['Index', 'Industry'], how='left')

# Final Step - Consolidation & Exporting  #

In [127]:
final_table

2,Index,Series Index Sep,Series Index Aug,Percent Point Change,Direction,Rate of Change,Trend* (Months),Series Index Sep,Series Index Aug,Percent Point Change
3,Services PMI®,53.6,54.5,-0.9,Growing,Slower,9,49.0,47.6,+1.4
4,Business Activity/ Production,58.8,57.3,+1.5,Growing,Faster,40,52.5,50.0,+2.5
5,New Orders,51.8,57.5,-5.7,Growing,Slower,9,49.2,46.8,+2.4
6,Employment,53.4,54.7,-1.3,Growing,Slower,4,51.2,48.5,+2.7
7,Supplier Deliveries,50.4,48.5,+1.9,Slowing,From Faster,1,46.4,48.6,-2.2
8,Inventories,54.2,57.7,-3.5,Growing,Slower,5,45.8,44.0,+1.8
9,Prices,58.9,58.9,0.0,Increasing,Same,76,43.8,48.4,-4.6
10,Backlog of Orders,48.6,41.8,+6.8,Contracting,Slower,2,42.4,44.1,-1.7
11,New Export Orders,63.7,62.1,+1.6,Growing,Faster,6,47.4,46.5,+0.9
12,Imports,50.6,52.3,-1.7,Growing,Slower,4,48.2,48.0,+0.2


In [128]:
commodity_up_df_split

,Commodity Price Up,Number of Month
1,Electrical Equipment,1
6,Labor — Contract,1
7,Labor — Services,1
9,Oil Related Products,1
3,Fuel,2
0,Diesel Fuel,3
2,Electrical Components,32
5,Labor,34
8,Lumber,4
4,Gasoline,8


In [129]:
commodity_down_df_split

,Commodities Down in Price,Number of Month
0,Steel Products (2).,1


In [130]:
commodity_short_df_split

,Commodities: Short Supply,Number of Month
0,Construction Materials,1
4,Labor — Construction,1
6,Vehicles,1
3,Labor,11
5,Transformers,13
1,Electrical Components,6
2,"Heating, Ventilation Air Conditioning",HVAC) Equipment (2


In [131]:
updated_index_data.tail(5)

,Date,NMI,Business Activity,New Order,Employment,Supplier Deliveries,Inventories,Prices,Backlog Orders,New Export Orders,Imports,Inventory Sentiment
310,2023-05-01,50.3,51.5,52.9,49.2,47.7,58.3,56.2,40.9,59.0,50.0,61.0
311,2023-06-01,53.9,59.2,55.5,53.1,47.6,55.9,54.1,43.9,61.5,54.6,54.0
312,2023-07-01,52.7,57.1,55.0,50.7,48.1,50.4,56.8,52.1,61.1,52.3,56.6
313,2023-08-01,54.5,57.3,57.5,54.7,48.5,57.7,58.9,41.8,62.1,52.3,61.5
314,2023-09-01,53.6,58.8,51.8,53.4,50.4,54.2,58.9,48.6,63.7,50.6,54.8


In [132]:
final_comment

,Industry,8/1/2023,2023-09-01
0,Accommodation & Food Services,"Restaurant sales and traffic trends remain positive year over year and compared to pre-pandemic (levels). Hiring is stable, with quality employees...","Prices are coming down across the board for most commodities. However, there are still a few areas where supply is not available on a consistent b..."
1,Construction,"Sales on a national level have been strong. Commodity material prices remain stable, and we are finding areas for cost reductions. Material availa...",Conditions remain favorable for mechanical contractors. New construction projects continue to launch. We are still seeing opportunities for cost r...
2,Health Care & Social Assistance,"While labor costs continue to soften, costs of pharmaceuticals and supplies remain stubbornly high, negatively impacting operating margins. Supply...",I think the outlook of our company and the industry is a slow but steady improvement — from a plethora of unknowns to a daily sense of being able ...
3,Management of Companies & Support Services,"The supply chain challenges affect a portion of our buys, as they include products and components made outside of the U.S. and are subject to ship...","Bank and leasing company volume seems to be falling as credit tightens, thus causing a slowdown in related services industries. Bankruptcy work is..."
4,Mining,Steady oil and gas production and sales volume. Declining commodity prices seem to have bottomed out.,NaN
5,"Professional, Scientific & Technical Services",The summer slowdown is similar to those in recent years due to vacations. Third-quarter projections are close to expectation. Inflationary costs a...,"The fourth quarter is looking better than forecast, which is good because the third quarter of 2023 was below plan. Our customers are cautiously o..."
6,Public Administration,Prices have settled. Warnings of a possible recession in 2024 are not being taken very seriously by top management. The same experts warned that t...,NaN
7,"Real Estate, Rental & Leasing","Overall conditions seem quite good, although there is definite slowdown in residential construction driven by rapidly increasing interest rates.",NaN
8,Retail Trade,"Business activity continues to be lower year over year, but we are meeting the year-to-date forecast.",Business is ramping up in preparation for the holiday season. Our supply chain is strong.
9,Utilities,Utility contractors in high demand.,"Other than increases in fuel costs and some materials, activity and sales have been relatively stable month over month. However, a seasonal decrea..."


In [133]:
sector_ranking_final

,Index,Industry,Translation,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01
0,NMI,"Agriculture, Forestry, Fishing & Hunting",农业、林业、渔业，狩猎业,-1.0,-2.0,-1.0,NaN
1,NMI,"Arts, Entertainment & Recreation",艺术，娱乐,2.0,-3.0,4.0,NaN
2,NMI,Accommodation & Food Services,住宿，食品服务,1.0,3.0,2.0,-3
3,NMI,Construction,建造,12.0,2.0,10.0,8
4,NMI,Educational Services,教育服务,5.0,11.0,9.0,13
5,NMI,Finance & Insurance,金融，保险,13.0,-4.0,11.0,7
6,NMI,Health Care & Social Assistance,医疗保健与社会救助,15.0,10.0,-4.0,6
7,NMI,Information,信息,-3.0,13.0,8.0,11
8,NMI,Management of Companies & Support Services,公司管理与支持服务,6.0,5.0,-5.0,-4
9,NMI,Mining,采矿,-2.0,-1.0,-2.0,3


In [469]:
# 1. Output below does not require data overrite 

final_table.to_excel('C:/Users/fanjz/Desktop/python/pmi/monthly_table.xlsx', index=False)
commodity_short_df_split.to_excel('C:/Users/fanjz/Desktop/python/pmi/commmodity_short.xlsx', index=False)
commodity_up_df_split.to_excel('C:/Users/fanjz/Desktop/python/pmi/commmodity_up.xlsx', index=False)
commodity_down_df_split.to_excel('C:/Users/fanjz/Desktop/python/pmi/commmodity_down.xlsx', index=False)

# 2. Output below requires data overrite over the existing file 

updated_index_data.to_excel('C:/Users/fanjz/Desktop/python/pmi/pmi_level.xlsx', index=False)
final_comment.to_excel('C:/Users/fanjz/Desktop/python/pmi/comments.xlsx', index=False)
sector_ranking_final.to_excel('C:/Users/fanjz/Desktop/python/pmi/sector_ranking.xlsx', index=False)